## Circuit_tool_ai_bot

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_CIRCUIT_API_KEY"))  # Load your OpenAI API key from environment variables

# Example form data
form_data = {
    "Name": "John Doe",
    "Service Number": "123456",
    "Claim Type": "Disability Compensation",
    "Details": "Veteran served in the U.S. Army from 2000 to 2005 and sustained injuries during service."
}

def generate_summary(form_data):
    prompt = f"Create a summary report for the following VA form data:\n{form_data}"
    try:
        # Call OpenAI's GPT model
        response = client.chat.completions.create(model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a summerizer assistant that only summerizes VA forms."},
            {"role": "user", "content": prompt}
        ],
        )
        # Extract and return the assistant's response
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
summary_report = generate_summary(form_data)
print("Summary Report:")
print(summary_report)

Summary Report:
Summary Report:

- Name: John Doe
- Service Number: 123456
- Claim Type: Disability Compensation
- Details: Veteran served in the U.S. Army from 2000 to 2005 and sustained injuries during service.


In [11]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import re
import spacy

# Ensure the spaCy model is installed
try:
    spacy.load("en_core_web_sm")
except OSError:
    # Install the model if not found
    !python -m spacy download en_core_web_sm
from base64 import b64encode

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_CIRCUIT_API_KEY"))

# Load spaCy for NLP (question detection)
nlp = spacy.load("en_core_web_sm")

def extract_text_from_image(image_path):
    """Extract text from image using OpenAI Vision API."""
    try:
        with open(image_path, "rb") as image_file:
            image_data = b64encode(image_file.read()).decode("utf-8")
        
        response = client.chat.completions.create(
            model="gpt-4-vision-preview",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "Extract text describing circuit parts (e.g., MOTOR, BOLT, O RING, CIRCUIT BREAKER) and their attributes (e.g., voltage, diameter, amps). Ignore irrelevant text."
                        },
                        {"type": "image_url", "image_url": f"data:image/jpeg;base64,{image_data}"}
                    ]
                }
            ],
            max_tokens=500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error processing image: {e}"

def is_question(text):
    """Detect if input is a question."""
    doc = nlp(text.lower())
    return any(token.text in ["what", "how", "why", "tell"] or token.tag_ == "WP" for token in doc)

def answer_question(text):
    """Provide general info for part-related questions."""
    text = text.lower()
    if "motor" in text:
        return ("A motor converts electrical energy to mechanical motion. Key specs include voltage (V), "
                "current (A), power (kW/HP), frequency (Hz), and phase. Can you share details about your motor?")
    elif "o ring" in text or "o-ring" in text:
        return ("An O-ring is a seal to prevent leaks, defined by outer diameter (OD), inner diameter (ID), "
                "and thickness (THK), e.g., 14MM OD X 10MM ID X 3MM THK. Do you have a specific O-ring?")
    elif "circuit breaker" in text:
        return ("A circuit breaker protects circuits from overloads, rated by amps, voltage, and poles, "
                "e.g., 10A, 400V, 2 POLE. Do you have a specific circuit breaker?")
    elif "bolt" in text:
        return ("A bolt is a fastener with a threaded shank, defined by type, diameter, and length, "
                "e.g., HEX, 2 IN DIA X 6 5/8 IN LG. Do you have a specific bolt?")
    return "I’m not sure what part you’re asking about. Can you clarify or provide an image/text?"

def parse_part(line):
    """Parse a single part line into formatted output."""
    line = line.strip().upper()
    
    # Regex patterns for parts
    motor_pattern = r"MOTOR,?\s*([\d.-/]+)?\s*(V|A|KW|HP|HZ|RPM|IP\d+|TYPE\s*[A-Z]|\d\s*PHASE)?"
    breaker_pattern = r"CIRCUIT BREAKER,?\s*([\d]+)\s*A,?\s*([\d/-]+)?\s*(V)?,?\s*(\d)?\s*(POLE)?"
    bolt_pattern = r"BOLT,?\s*([A-Z]+)?,?\s*([\d\s/]+)\s*IN\s*DIA\s*X\s*([\d\s/]+)?\s*IN\s*LG"
    oring_pattern = r"O RING,?\s*(\w+)?,?\s*([\d]+)\s*MM\s*OD\s*X\s*([\d]+)\s*MM\s*ID\s*X\s*([\d\s/]+)\s*MM\s*THK"
    
    if "MOTOR" in line:
        attributes = re.findall(motor_pattern, line)
        attrs = {v.split()[1]: v.split()[0] for v in attributes if v[0]}
        formatted = [
            attrs.get("A", "AMPS UNKNOWN"),
            attrs.get("V", "VOLTS UNKNOWN"),
            attrs.get("KW", attrs.get("HP", "POWER UNKNOWN")),
            attrs.get("HZ", "HZ UNKNOWN"),
            attrs.get("PHASE", "3 PHASE"),
            attrs.get("IP", ""),
            attrs.get("RPM", ""),
            attrs.get("TYPE", "")
        ]
        formatted = [x for x in formatted if x]
        prompt = ("I see a motor. Can you provide missing details like "
                  f"{'current (amps)' if 'AMPS UNKNOWN' in formatted else ''} "
                  f"{'power (kW/HP)' if 'POWER UNKNOWN' in formatted else ''} "
                  f"{'frequency (Hz)' if 'HZ UNKNOWN' in formatted else ''}?").strip()
        return f"MOTOR, {', '.join(formatted)}", prompt
    
    elif "CIRCUIT BREAKER" in line:
        match = re.match(breaker_pattern, line)
        if match:
            volts = f"{match.group(2)}V" if match.group(2) else "VOLTS UNKNOWN"
            poles = f"{match.group(4)} POLE" if match.group(4) else "POLES UNKNOWN"
            prompt = ("Can you provide " +
                      f"{'voltage' if volts == 'VOLTS UNKNOWN' else ''} " +
                      f"{'poles' if poles == 'POLES UNKNOWN' else ''}?").strip()
            return f"CIRCUIT BREAKER, {match.group(1)}A, {volts}, {poles}", prompt
    
    elif "BOLT" in line:
        match = re.match(bolt_pattern, line)
        if match:
            length = match.group(3) or "LENGTH UNKNOWN"
            prompt = "Can you provide the bolt’s length?" if length == "LENGTH UNKNOWN" else ""
            return f"BOLT, {match.group(1) or ''}, {match.group(2)} IN DIA X {length}".strip(", "), prompt
    
    elif "O RING" in line:
        match = re.match(oring_pattern, line)
        if match:
            material = match.group(1) or ""
            return f"O RING, {material}, {match.group(2)}MM OD X {match.group(3)}MM ID X {match.group(4)}MM THK".strip(", "), ""
    
    return f"UNRECOGNIZED: {line}", "I don’t recognize this part. Can you provide more details or an image?"

def process_input(image_path=None, text=None):
    """Process image or text input and return formatted parts list with prompts."""
    try:
        if image_path:
            text = extract_text_from_image(image_path)
        if not text:
            return [("No input provided", "Please upload an image or provide text describing a part.")]
        
        lines = text.split("\n")
        results = []
        for line in lines:
            line = line.strip()
            if not line:
                continue
            if is_question(line):
                results.append((answer_question(line), ""))
            else:
                part, prompt = parse_part(line)
                results.append((part, prompt))
        return results
    except Exception as e:
        return [(f"Error processing input: {e}", "Please try again or provide a different image/text.")]

# Example usage
image_input = "circuit_image.jpg"  # Replace with actual image path
text_input = """
MOTOR, 200-230V, TYPE P
What’s an O-ring?
BOLT, 2 IN DIA, HEX
"""

# Test with text input
print("Processing text input:")
results = process_input(text=text_input)
for part, prompt in results:
    print(f"Result: {part}")
    if prompt:
        print(f"Prompt: {prompt}")

# Test with image input (uncomment and provide valid image path)
# print("\nProcessing image input:")
# results = process_input(image_path=image_input)
# for part, prompt in results:
#     print(f"Result: {part}")
#     if prompt:
#         print(f"Prompt: {prompt}")

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.3 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Processing text input:
Result: Error processing input: 'tuple' object has no attribute 'split'
Prompt: Please try again or provide a different image/text.
